In [1]:
import json

with open('data/doctor.json','r') as f:
    doctors = json.load(f)
    
patients = []
with open('data/patient.json') as f:
    for line in f:
        patients.append(json.loads(line))

print(len(doctors),len(patients))

958 187788


In [28]:
# print(patients[5]['telecom'][1]['value'])
patients[122]

{'resourceType': 'Patient',
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"/><table class="hapiPropertyTable"><tbody><tr><td>Identifier</td><td>123456789123</td></tr><tr><td>Address</td><td><span>Cao Bằng </span></td></tr></tbody></table></div>'},
 'extension': [{'url': 'http://vinbrain.net/Patient#YearOfBirth',
   'valueInteger': 1968},
  {'url': 'http://vinbrain.net/Patient#EducationLevel',
   'valueString': 'PRIMARY_SCHOOL'},
  {'url': 'http://vinbrain.net/Patient#Income',
   'valueString': 'LESS_THAN_TWO'},
  {'url': 'http://vinbrain.net/Patient#DiseaseDiagnosis',
   'valueCodeableConcept': {'coding': [{'code': 'HYPERTENSION_FILTER_NEGATIVE'}]}},
  {'url': 'http://vinbrain.net/Patient#DiseaseDiagnosis',
   'valueCodeableConcept': {'coding': [{'code': 'DIABETES_FILTER_NEGATIVE'}]}},
  {'url': 'http://vinbrain.net/Patient#DiseaseResult',
   'valueCodeableConcept': {'coding': [{'code': 'HYPERTENSION_UNMEDICATED'}]}},
  {

In [4]:
patients[1231]['address']

[{'text': 'Xóm Mỏ Sắt, Dân Chủ', 'city': 'Cao Bằng', 'district': 'Hoà An'}]

## build csv data

In [5]:
age = 0
income = 0
gender = 0
address = 0

for pa in patients:
    try:
        extension = pa['extension'] # list
    except:
        continue
        
    for item in extension:
        if item['url'] == 'http://vinbrain.net/Patient#YearOfBirth':
            age += 1
        if item['url'] == 'http://vinbrain.net/Patient#Income':
            income += 1
    if 'gender' in list(pa.keys()):
        gender +=1
    if 'address' in list(pa.keys()):
        address +=1 
        
print(age/len(patients)) # 0.03382005239951435
print(income/len(patients)) # 0.09373868404796899
print(gender/len(patients)) # 0.8780166996826209
print(address/len(patients)) # 0.9097705923701195

0.03382005239951435
0.09373868404796899
0.8780166996826209
0.9097705923701195


In [7]:
print(len(patients))
print(age,income,gender,address)

187788
6351 17603 164881 170844


In [76]:
def select_patient_income(pa):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Patient#Income':
                return item['valueString']
        return None
    except:
        return None
            
def select_patient_gender(pa):
    if 'gender' in (pa.keys()):
        return pa['gender']
    return None

def select_patient_age(pa):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Patient#YearOfBirth':
                return item['valueInteger']
        
        if 'birthDate' in list(pa.keys()):
            return pa['birthDate']
        
        return None
    except:
        return None

def select_patient_address(pa):
    if 'address' in list(pa.keys()):
        return pa['address']
    return None

def select_patient_name(pa):
    if 'name' in list(pa.keys()):
        return pa['name'][0]['text']
    return None

In [77]:
def is_clean_patient_data(income,gender,age,address,name):
    num_na = 0
    if income == None:
        num_na += 1
    if gender == None:
        num_na += 1
    if age == None:
        num_na += 1
    if address == None:
        num_na += 1
    
    if num_na == 4:
        return False

    return True

In [78]:
from datetime import date

def normalize_patient_age(age):
    if age == None:
        return None
    if type(age) == int:
        return age
    
    age = age.split('-')[0]
    return age

In [79]:
import pandas as pd
from tqdm import tqdm

PATIENT_ID = []
PATIENT_INCOME = []
PATIENT_GENDER = []
PATIENT_AGE = []
PATIENT_ADDRESS = []
PATIENT_NAME = []

for id,pa in tqdm(enumerate(patients)):
    income = select_patient_income(pa)
    gender = select_patient_gender(pa)
    age = select_patient_age(pa)
    age = normalize_patient_age(age)
    address = select_patient_address(pa)
    name = select_patient_name(pa)

    if is_clean_patient_data(income,gender,age,address,name):
        PATIENT_INCOME.append(income)
        PATIENT_GENDER.append(gender)
        PATIENT_AGE.append(age)
        PATIENT_ADDRESS.append(address)
        PATIENT_NAME.append(name)
        PATIENT_ID.append(id)
df = pd.DataFrame({
    'id' : PATIENT_ID,
    'income':PATIENT_INCOME,
    'gender':PATIENT_GENDER,
    'age':PATIENT_AGE,
    'address':PATIENT_ADDRESS,
    'name':PATIENT_NAME,
})

print(len(patients))
df = df.dropna(how='all') # drop row where all fill are NA

print(len(df))
df.to_csv('patient_data.csv',",",index=False)

187788it [00:00, 279926.65it/s]


187788
170844


In [53]:
patients[45]

{'resourceType': 'Patient',
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"/><table class="hapiPropertyTable"><tbody><tr><td>Identifier</td><td>97b33942-0196-4642-bd2d-de9a9e432b67</td></tr><tr><td>Address</td><td><span>Cao Bằng </span></td></tr></tbody></table></div>'},
 'extension': [{'url': 'http://vinbrain.net/Patient#YearOfBirth',
   'valueInteger': 1958},
  {'url': 'http://vinbrain.net/Patient#EducationLevel',
   'valueString': 'SECONDARY_SCHOOL'},
  {'url': 'http://vinbrain.net/Patient#Income', 'valueString': 'TWO_TO_FOUR'},
  {'url': 'http://vinbrain.net/Patient#DiseaseDiagnosis',
   'valueCodeableConcept': {'coding': [{'code': 'HYPERTENSION_FILTER_NEGATIVE'}]}},
  {'url': 'http://vinbrain.net/Patient#DiseaseDiagnosis',
   'valueCodeableConcept': {'coding': [{'code': 'DIABETES_FILTER_NEGATIVE'}]}},
  {'url': 'http://vinbrain.net/Patient#DiseaseResult',
   'valueCodeableConcept': {'coding': [{'code': 'HYPERTENSION_

In [71]:
doctors[220]

{'resourceType': 'Practitioner',
 'extension': [{'url': 'http://vinbrain.net/Practitioner#FirebaseTokens',
   'valueCodeableConcept': {'coding': [{'code': 'f1jd5XI2cKxCiOqsVfj8d0:APA91bHegb7CM_uE4F0G46fORE3GJ4K5VyDi66w1cZVsqJ3FmGUplw_K-oYX3zWmdQAp3AGlHU8zcH0SdKz8NSTljSzXfZq5gxrJxD-OdJ5lI6vuUybYZkXJvtez55L5P77CJGC5BD4g'},
     {'code': 'cedwvFFB92uPZORTBa6sph:APA91bHtEXGLkODI-JpnAzdll99WMkf0pXsBPvoJL8Y3QPcov2G1ELUXODiT_TiEkj87Mbe1uHR8yqCD8r-eHVVChhq_ClC_ZF_pA6TY30bsr4iNKuW72I_nWKsyEk2ca6mSMBzd7lOw'},
     {'code': 'cedwvFFB92uPZORTBa6sph:APA91bGJLXioNC4kzDwIHnNfctjH5SqPW3f02QyRfqB5l2ZzKKeohMpRgZ4Fmg2_K0Nj8p4jCQONno70fLCbCjM31TCu7X_qQb9LrUwuHCa44CZ0KESLK9khanPYpfMulNJxmRUWzF-D'},
     {'code': 'cedwvFFB92uPZORTBa6sph:APA91bFN_UqWKjAI3KEUrD2e1exNekNUJ0Fyl5lKvCiXJySZLD2nj42bcsZsQBJM391eL87uExy7sxJtmiqTWQ8QEprYl2HSQqYHENEnUPIeHgT8kQBNBoW7wpUrFdsDoNHTx-NSpwKq'}]}},
  {'url': 'http://vinbrain.net/Practitioner#Department',
   'valueString': 'Doctor'},
  {'url': 'http://vinbrain.net/Practitione

In [73]:
dept_keys = {
    'SP001':'bác sĩ gia đình',
    'SP002':'da liễu',
    'SP003':'dinh dưỡng',
    'SP004':'hô hấp',
    'SP005':'nội thận',
    'SP006':'nhi',
    'SP007':'nội tiết',
    'SP008':'nội tiêu hóa',
    'SP009':'nội tổng quát',
    'SP010':'tim mạch',
    'SP011':'tâm thần',
    'SP012':'thần kinh',
    'SP013':'tuyến vú',
    'SP014':'chẩn đoán hình ảnh',
    'SP015':'chấn thương chỉnh hình',
    'SP016':'nhiễm',
    'SP017':'sản phụ khoa',
    'SP018':'tai mũi họng',
}

In [74]:
def select_speciality(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Practitioner#Specialties':
                return dept_keys[item['valueCodeableConcept']['coding'][0]['code']]
        return None
    except:
        return None

def select_title(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Practitioner#Education':
                return item['valueCodeableConcept']['coding'][0]['code']
        return None
    except:
        return None
    
def select_rating(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Practitioner#Rating':
                return item['valueDecimal']
        return None
    except:
        return None

def select_reviewer(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Practitioner#Reviewers':
                return item['valueInteger']
        return None
    except:
        return None
    
def select_hourly_rate(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == 'http://vinbrain.net/Practitioner#HourlyRate':
                return item['valueDecimal']
        return None
    except:
        return None

def select_experience(doctor):
    try:
        extension = pa['extension'] # list
        for item in extension:
            if item['url'] == "http://vinbrain.net/Practitioner#Experience":
                return item['valueString']
        return None
    except:
        return None
    

In [75]:
'''
- title
- rating
- hourly rate
- experience # khong tim thay
- review
- age
- gender
'''

DOCTOR_ID = []
DOCTOR_TITLE = []
DOCTOR_RATING = []
DOCTOR_AGE = []
DOCTOR_NAME = []
DOCTOR_REVIWER = []
DOCTOR_EXPERIENCE = []
DOCTOR_HOURLY_RATE = []
DOCTOR_GENDER = []
DOCTOR_SPECIALITY = []

for id,pa in tqdm(enumerate(doctors)):
    gender = select_patient_gender(pa)
    age = select_patient_age(pa)
    age = normalize_patient_age(age)
    name = select_patient_name(pa)
    
    speciality = select_speciality(pa)
    title = select_title(pa)
    rating = select_rating(pa)
    review = select_reviewer(pa)
    hourly_rate = select_hourly_rate(pa)
    experience = select_experience(pa)
    
    DOCTOR_GENDER.append(gender)
    DOCTOR_AGE.append(age)
    DOCTOR_NAME.append(name)
    DOCTOR_ID.append(id)

    DOCTOR_SPECIALITY.append(speciality)
    DOCTOR_REVIWER.append(review)
    DOCTOR_HOURLY_RATE.append(hourly_rate)
    DOCTOR_TITLE.append(title)
    DOCTOR_EXPERIENCE.append(experience)
    
df = pd.DataFrame({
    'id' : DOCTOR_ID,
    'gender':DOCTOR_GENDER,
    'age':DOCTOR_AGE,
    'name':DOCTOR_NAME,
    'title': DOCTOR_TITLE,
    'speciality':DOCTOR_SPECIALITY,
    'review':DOCTOR_REVIWER,
    'hourly_rate':DOCTOR_HOURLY_RATE,
    'experience':DOCTOR_EXPERIENCE
})

print(len(doctors))
df = df.dropna(how='all') # drop row where all fill are NA

print(len(df))
df.to_csv('doctor_data.csv',",",index=False)

958it [00:00, 160571.58it/s]

958
958


## Stats user's email existence

In [27]:
from validate_email_address import validate_email
from tqdm import tqdm

not_exists = 0
total_email = 0
for i, pa in tqdm(enumerate(patients)):
    try:
        email = pa['telecom'][1]['value']
        total_email += 1
    except:
        continue
        
    try:
        isExists = validate_email(email, verify=True)
        if isExists == None:
            not_exists += 1
    except:
        not_exists += 1
        continue
    
    if i % 10000 == 0:
        print('Not existed email',not_exists)
        print('% not existed email',(not_exists/total_email))

100%|██████████| 187788/187788 [20:19<00:00, 153.94it/s]  


In [28]:
print('Not existed email',not_exists)
print('% not existed email',(not_exists/total_email))

Not existed email 137
% not existed email 0.04537926465717125


In [29]:
total_email

3019